In [1]:
!pip install accelerate transformers==4.28.0 datasets evaluate rouge_score > /dev/null

In [2]:
import pandas as pd
from datasets import load_dataset
from datasets import Dataset, DatasetDict

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [19]:
!wget https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv

--2023-06-07 06:38:56--  https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027887 (1004K) [text/plain]
Saving to: ‘1k.csv’

1k.csv              100%[===================>]   1004K  --.-KB/s    in 0.1s    

2023-06-07 06:38:57 (9.49 MB/s) - ‘1k.csv’ saved [1027887/1027887]



In [20]:
one_Kdata = pd.read_csv("/content/1k.csv")
one_Kdata.shape

(998, 2)

In [34]:
one_Kdata = one_Kdata.loc[one_Kdata.prompt.str.split(' ').str.len() > 6]

In [35]:
one_Kdata['seq1'] = one_Kdata.prompt.apply(lambda x: " ".join(x.split(' ')[:4]))
one_Kdata['seq2'] = one_Kdata.prompt.apply(lambda x: " ".join(x.split(' ')[4:]))

In [36]:
one_Kdata.head(5)

,prompt,raw_data,seq1,seq2
0,A portrait photo of a kangaroo wearing an oran...,"{""image_uri"": ""PENDING"", ""modifiers"": [""portra...",A portrait photo of,a kangaroo wearing an orange hoodie and blue s...
1,inmates with cow heads inside a jailcell,"{""image_uri"": ""PENDING"", ""modifiers"": [""inmate...",inmates with cow heads,inside a jailcell
2,"daguerrotype of a corgi astronaut on the moon,...","{""image_uri"": ""PENDING"", ""modifiers"": [""daguer...",daguerrotype of a corgi,"astronaut on the moon, award - winning photogr..."
3,totem animal tribal chaman vodoo mask feather ...,"{""image_uri"": ""PENDING"", ""modifiers"": [""totem ...",totem animal tribal chaman,vodoo mask feather gemstone plant wood rock vi...
5,Illustration of a sugar skull day of the dead ...,"{""image_uri"": ""PENDING"", ""modifiers"": [""illust...",Illustration of a sugar,"skull day of the dead girl, art by christopher..."


In [40]:
one_KData = one_Kdata.dropna()
one_Kdata.drop(['raw_data',"prompt"],
               axis=1,
               inplace=True)

In [52]:
train_oneK = one_Kdata[:700]
test_oneK = one_Kdata[700:]

In [53]:
test_oneK.head()

,seq1,seq2
775,"tower of babel doom,","hell on earth, Karlkka, trending on Artstation..."
776,fantasy art of doom,"slayer, intricate, high detailed, insane detai..."
777,"exquisite creature poster art,","fun, vibrant, smooth, elegant, by weta studio ..."
778,portrait of a man,"by greg rutkowski, he looks like tye sheridan,..."
779,wednesday addams as death,"of the endless, the sandman, made by carvaggio..."


In [54]:
krea_train = Dataset.from_pandas(train_oneK,split='train')

In [55]:
krea_test = Dataset.from_pandas(test_oneK,split='test')

In [56]:
krea = DatasetDict()
 
krea['train'] = krea_train 
krea['test'] = krea_test

In [ ]:
krea.push_to_hub("Kamaljp/krea-openprompt")

In [58]:
krea['train'][:5]

{'seq1': ['A portrait photo of',
  'inmates with cow heads',
  'daguerrotype of a corgi',
  'totem animal tribal chaman',
  'Illustration of a sugar'],
 'seq2': ['a kangaroo wearing an orange hoodie and blue sunglasses standing on the grass in front of the Sydney Opera House holding a sign on the chest that says Welcome Friends, subject: kangaroo, subject detail: wearing orange hoodie, wearing blue sunglasses, subject location: sydney opera house, subject action: holding sign',
  'inside a jailcell',
  'astronaut on the moon, award - winning photograph, vintage, stunning',
  'vodoo mask feather gemstone plant wood rock video game illustration vivid color borderlands by josan gonzales and dan mumford radiating a glowing aura',
  'skull day of the dead girl, art by christopher vacher'],
 '__index_level_0__': [0, 1, 2, 3, 5]}

In [59]:
from evaluate import load

metric = load('rouge')

In [7]:
import random
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
model_checkpoint = "t5-small"

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [60]:
max_input_length = 1024
max_target_length = 128
prefix = 'Write about '
def preprocess_function(examples):
  #print(len(examples['prompt']))
  inputs = [prefix + doc for doc in examples["seq1"]]
  model_inputs = tokenizer(inputs, 
                             max_length=max_input_length, 
                             truncation=True)
  # Setup the tokenizer for targets
  labels = tokenizer(text_target=examples["seq2"], 
                     max_length=max_target_length, 
                     truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [61]:
tokenized_krea = krea.map(preprocess_function,batched=True)

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/194 [00:00<?, ? examples/s]

In [62]:
from transformers import (AutoModelForSeq2SeqLM, 
                          DataCollatorForSeq2Seq, 
                          Seq2SeqTrainingArguments, 
                          Seq2SeqTrainer)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [80]:
batch_size = 16

model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3000,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [81]:
data_collator = DataCollatorForSeq2Seq(tokenizer, 
                                       model=model)

In [65]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, 
                                           skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, 
                      labels, 
                      tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, 
                                            skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, 
                            references=decoded_labels, 
                            use_stemmer=True, 
                            use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [82]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_krea["train"],
    eval_dataset=tokenized_krea["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/t5-small-finetuned-xsum is already a clone of https://huggingface.co/Kamaljp/t5-small-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.


In [69]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [83]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,5.167398,2.893900,0.309700,2.706800,2.750000,18.072200
2,No log,4.959458,1.732800,0.113500,1.663600,1.689400,18.969100
3,No log,4.798594,3.248800,0.229300,3.116400,3.134900,18.969100
4,No log,4.682513,3.650200,0.282100,3.548800,3.580000,18.979400
5,No log,4.586305,3.862200,0.243700,3.793200,3.797800,18.979400
6,No log,4.506121,4.125200,0.288000,4.033300,4.036600,19.000000
7,No log,4.443467,3.701400,0.157400,3.637000,3.638800,19.000000
8,No log,4.388759,4.095700,0.225900,3.995200,4.021200,19.000000
9,No log,4.342234,3.998600,0.216100,3.925700,3.950300,19.000000
10,No log,4.302193,4.047000,0.216100,3.992400,4.048400,19.000000


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1993 in _inner_training_loop     │
│                                                                                                  │
│   1990 │   │   │   │   │   │   scale_before = self.scaler.get_scale()                            │
│   1991 │   │   │   │   │   │   self.scaler.step(self.optimizer)                                  │
│   1992 │   │   │   │   │   │   self.scaler.update()                                              │
│ ❱ 1993 │   │   │   │   │   │   scale_after = self.scaler.get_scale()                             │
│   1994 │   │   │   │   │   │   optimizer_was_run = scale_before <= scale_after                   │
│   1995 │   │   │   │   │   else:                                                                 │
│   1996 │   │   │   │   │   │   self.optimizer.step()                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:450 in get_scale           │
│                                                                                                  │
│   447 │   │   │   :meth:`get_scale` incurs a CPU-GPU sync.                                       │
│   448 │   │   """                                                                                │
│   449 │   │   if self._enabled:                                                                  │
│ ❱ 450 │   │   │   return self._init_scale if self._scale is None else self._get_scale_async().   │
│   451 │   │   else:                                                                              │
│   452 │   │   │   return 1.0                                                                     │
│   453                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [84]:
trainer.push_to_hub("krea_tunedT5")

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/Jun07_07-29-41_4c3084dc9254/events.out.tfevents.1686122996.4c3084dc9254.443.5:   0%|         …

To https://huggingface.co/Kamaljp/t5-small-finetuned-xsum
   ab8bcc0..e81db10  main -> main

   ab8bcc0..e81db10  main -> main

To https://huggingface.co/Kamaljp/t5-small-finetuned-xsum
   e81db10..359db9c  main -> main

   e81db10..359db9c  main -> main



'https://huggingface.co/Kamaljp/t5-small-finetuned-xsum/commit/e81db10afbe3a9879e9d2fa9e24cf3cd0e1a97b4'

In [85]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("Kamaljp/t5-small-finetuned-xsum")

In [86]:
text = "totem animal tribal chaman"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to("cpu")

In [87]:
outputs = model.generate(**inputs, max_new_tokens=200,
                         be)

In [88]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

a saline saline saline saline s
